In [268]:
# import libraries

import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import mysql.connector
import pymysql
pymysql.install_as_MySQLdb()

In [218]:
# import dataset

rioja_meteo = pd.read_csv('../data/Logrono_Rioja_meteo.csv')

In [219]:
rioja_meteo

,latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation
0,42.5,-2.5,442.0,7200.0,Europe/Berlin,CEST
1,time,temperature_2m_mean (°C),precipitation_sum (mm),NaN,NaN,NaN
2,2013-01-01,7.0,5.60,NaN,NaN,NaN
3,2013-01-02,6.0,1.40,NaN,NaN,NaN
4,2013-01-03,5.2,0.00,NaN,NaN,NaN
...,...,...,...,...,...,...
2919,2020-12-27,4.5,1.00,NaN,NaN,NaN
2920,2020-12-28,6.0,8.30,NaN,NaN,NaN
2921,2020-12-29,5.0,0.10,NaN,NaN,NaN
2922,2020-12-30,4.5,0.00,NaN,NaN,NaN


In [220]:
# drop first two rows

rioja_meteo.drop(rioja_meteo.index[0:2], inplace=True)
rioja_meteo

,latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation
2,2013-01-01,7.0,5.60,NaN,NaN,NaN
3,2013-01-02,6.0,1.40,NaN,NaN,NaN
4,2013-01-03,5.2,0.00,NaN,NaN,NaN
5,2013-01-04,4.9,0.00,NaN,NaN,NaN
6,2013-01-05,5.5,0.00,NaN,NaN,NaN
...,...,...,...,...,...,...
2919,2020-12-27,4.5,1.00,NaN,NaN,NaN
2920,2020-12-28,6.0,8.30,NaN,NaN,NaN
2921,2020-12-29,5.0,0.10,NaN,NaN,NaN
2922,2020-12-30,4.5,0.00,NaN,NaN,NaN


In [221]:
rioja_meteo = rioja_meteo.reset_index()
rioja_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2922 entries, 0 to 2921
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  2922 non-null   int64  
 1   latitude               2922 non-null   object 
 2   longitude              2922 non-null   object 
 3   elevation              2922 non-null   object 
 4   utc_offset_seconds     0 non-null      float64
 5   timezone               0 non-null      object 
 6   timezone_abbreviation  0 non-null      object 
dtypes: float64(1), int64(1), object(5)
memory usage: 159.9+ KB


In [222]:
# drop columns

rioja_meteo.drop(columns = ['index', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation'], inplace=True)
rioja_meteo

,latitude,longitude,elevation
0,2013-01-01,7.0,5.60
1,2013-01-02,6.0,1.40
2,2013-01-03,5.2,0.00
3,2013-01-04,4.9,0.00
4,2013-01-05,5.5,0.00
...,...,...,...
2917,2020-12-27,4.5,1.00
2918,2020-12-28,6.0,8.30
2919,2020-12-29,5.0,0.10
2920,2020-12-30,4.5,0.00


In [223]:
# rename columns

rioja_meteo.rename(columns={'latitude': 'date', 'longitude': 'temp_rioja', 'elevation': 'precip_rioja'}, inplace=True)
rioja_meteo

,date,temp_rioja,precip_rioja
0,2013-01-01,7.0,5.60
1,2013-01-02,6.0,1.40
2,2013-01-03,5.2,0.00
3,2013-01-04,4.9,0.00
4,2013-01-05,5.5,0.00
...,...,...,...
2917,2020-12-27,4.5,1.00
2918,2020-12-28,6.0,8.30
2919,2020-12-29,5.0,0.10
2920,2020-12-30,4.5,0.00


In [224]:
# date: change type into string
# temp, precipitation: change type into float

rioja_meteo.date = rioja_meteo.date.apply(lambda x: str(x))
rioja_meteo.temp_rioja = rioja_meteo.temp_rioja.apply(lambda x: float(x))
rioja_meteo.precip_rioja = rioja_meteo.precip_rioja.apply(lambda x: float(x))

rioja_meteo

,date,temp_rioja,precip_rioja
0,2013-01-01,7.0,5.6
1,2013-01-02,6.0,1.4
2,2013-01-03,5.2,0.0
3,2013-01-04,4.9,0.0
4,2013-01-05,5.5,0.0
...,...,...,...
2917,2020-12-27,4.5,1.0
2918,2020-12-28,6.0,8.3
2919,2020-12-29,5.0,0.1
2920,2020-12-30,4.5,0.0


In [225]:
rioja_meteo['date'].astype('string')

0       2013-01-01
1       2013-01-02
2       2013-01-03
3       2013-01-04
4       2013-01-05
           ...    
2917    2020-12-27
2918    2020-12-28
2919    2020-12-29
2920    2020-12-30
2921    2020-12-31
Name: date, Length: 2922, dtype: string

In [226]:
rioja_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2922 entries, 0 to 2921
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          2922 non-null   object 
 1   temp_rioja    2922 non-null   float64
 2   precip_rioja  2922 non-null   float64
dtypes: float64(2), object(1)
memory usage: 68.6+ KB


In [227]:
# group data by month (regex extract characters in date)
# mean temperature
# sum precipitation

# join on 'data' in sql, adding all 3 regions (in the end I did concat directly in python)


In [228]:
# regex extract year+month in date

rioja_meteo['date'] = rioja_meteo['date'].str.extract('(.{7})')
rioja_meteo


,date,temp_rioja,precip_rioja
0,2013-01,7.0,5.6
1,2013-01,6.0,1.4
2,2013-01,5.2,0.0
3,2013-01,4.9,0.0
4,2013-01,5.5,0.0
...,...,...,...
2917,2020-12,4.5,1.0
2918,2020-12,6.0,8.3
2919,2020-12,5.0,0.1
2920,2020-12,4.5,0.0


In [229]:
# mean temperature, grouped by month

rioja_temp = rioja_meteo.groupby(by="date", as_index=False).temp_rioja.mean()
rioja_temp

,date,temp_rioja
0,2013-01,5.690323
1,2013-02,4.760714
2,2013-03,7.932258
3,2013-04,9.703333
4,2013-05,9.938710
...,...,...
91,2020-08,21.722581
92,2020-09,18.906667
93,2020-10,12.641935
94,2020-11,9.886667


In [230]:
# sum precipitation, grouped by month

rioja_precip = rioja_meteo.groupby(by='date', as_index=False).precip_rioja.sum()
rioja_precip

,date,precip_rioja
0,2013-01,126.9
1,2013-02,89.2
2,2013-03,132.2
3,2013-04,57.3
4,2013-05,86.0
...,...,...
91,2020-08,20.8
92,2020-09,27.1
93,2020-10,61.4
94,2020-11,39.7


In [231]:
# concat temp + precip

rioja = pd.concat([rioja_temp, rioja_precip], axis=1)
rioja

,date,temp_rioja,date,precip_rioja
0,2013-01,5.690323,2013-01,126.9
1,2013-02,4.760714,2013-02,89.2
2,2013-03,7.932258,2013-03,132.2
3,2013-04,9.703333,2013-04,57.3
4,2013-05,9.938710,2013-05,86.0
...,...,...,...,...
91,2020-08,21.722581,2020-08,20.8
92,2020-09,18.906667,2020-09,27.1
93,2020-10,12.641935,2020-10,61.4
94,2020-11,9.886667,2020-11,39.7


In [232]:
# rename date to month

rioja.rename(columns={'date': 'month'}, inplace=True)
rioja

,month,temp_rioja,month,precip_rioja
0,2013-01,5.690323,2013-01,126.9
1,2013-02,4.760714,2013-02,89.2
2,2013-03,7.932258,2013-03,132.2
3,2013-04,9.703333,2013-04,57.3
4,2013-05,9.938710,2013-05,86.0
...,...,...,...,...
91,2020-08,21.722581,2020-08,20.8
92,2020-09,18.906667,2020-09,27.1
93,2020-10,12.641935,2020-10,61.4
94,2020-11,9.886667,2020-11,39.7


In [233]:
rioja.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   month         96 non-null     object 
 1   temp_rioja    96 non-null     float64
 2   month         96 non-null     object 
 3   precip_rioja  96 non-null     float64
dtypes: float64(2), object(2)
memory usage: 3.1+ KB


In [234]:
# load clean tables from other two regions

catalunya = pd.read_csv('../data/catalunya_clean.csv')

In [235]:
duero = pd.read_csv('../data/duero_clean.csv')

In [236]:
# concat all tree regions together

tree_regions = pd.concat([rioja, catalunya, duero], axis=1)
tree_regions

,month,temp_rioja,month,precip_rioja,temp_catalunya,precip_catalunya,temp_duero,precip_duero
0,2013-01,5.690323,2013-01,126.9,7.509677,25.0,4.429032,74.7
1,2013-02,4.760714,2013-02,89.2,7.310714,58.1,3.450000,60.2
2,2013-03,7.932258,2013-03,132.2,10.283871,119.3,6.222581,138.8
3,2013-04,9.703333,2013-04,57.3,12.816667,86.2,8.890000,60.0
4,2013-05,9.938710,2013-05,86.0,14.525806,47.4,9.951613,59.1
...,...,...,...,...,...,...,...,...
91,2020-08,21.722581,2020-08,20.8,25.277419,31.1,22.454839,37.5
92,2020-09,18.906667,2020-09,27.1,20.956667,22.4,18.440000,32.7
93,2020-10,12.641935,2020-10,61.4,15.280645,17.2,11.332258,73.1
94,2020-11,9.886667,2020-11,39.7,12.043333,76.3,9.340000,18.1


In [237]:
# save clean dataset to tree_regions_clean.csv

tree_regions.to_csv("../data/tree_regions_clean.csv", index=False)

In [238]:
# test if the file was saved correctly, by loading it

tree_regions_clean = pd.read_csv("../data/tree_regions_clean.csv")

In [239]:
# extract columns 'month' and 'month.1' -> one to year, and the other to month

tree_regions_clean['month'] = tree_regions_clean['month'].str.extract('(.{4})')
tree_regions_clean['month.1'] = tree_regions_clean['month.1'].str.extract('(.{2}$)')

tree_regions_clean

,month,temp_rioja,month.1,precip_rioja,temp_catalunya,precip_catalunya,temp_duero,precip_duero
0,2013,5.690323,01,126.9,7.509677,25.0,4.429032,74.7
1,2013,4.760714,02,89.2,7.310714,58.1,3.450000,60.2
2,2013,7.932258,03,132.2,10.283871,119.3,6.222581,138.8
3,2013,9.703333,04,57.3,12.816667,86.2,8.890000,60.0
4,2013,9.938710,05,86.0,14.525806,47.4,9.951613,59.1
...,...,...,...,...,...,...,...,...
91,2020,21.722581,08,20.8,25.277419,31.1,22.454839,37.5
92,2020,18.906667,09,27.1,20.956667,22.4,18.440000,32.7
93,2020,12.641935,10,61.4,15.280645,17.2,11.332258,73.1
94,2020,9.886667,11,39.7,12.043333,76.3,9.340000,18.1


In [240]:
# rename columns

tree_regions_clean.rename(columns={'month': 'year', 'month.1': 'month', 'temp_rioja': 'temp_rioja_mean', 'temp_catalunya': 'temp_catalunya_mean', 'temp_duero': 'temp_duero_mean', 'precip_rioja': 'precip_rioja_sum', 'precip_catalunya': 'precip_catalunya_sum', 'precip_duero': 'precip_duero_sum'}, inplace=True)
tree_regions_clean

,year,temp_rioja_mean,month,precip_rioja_sum,temp_catalunya_mean,precip_catalunya_sum,temp_duero_mean,precip_duero_sum
0,2013,5.690323,01,126.9,7.509677,25.0,4.429032,74.7
1,2013,4.760714,02,89.2,7.310714,58.1,3.450000,60.2
2,2013,7.932258,03,132.2,10.283871,119.3,6.222581,138.8
3,2013,9.703333,04,57.3,12.816667,86.2,8.890000,60.0
4,2013,9.938710,05,86.0,14.525806,47.4,9.951613,59.1
...,...,...,...,...,...,...,...,...
91,2020,21.722581,08,20.8,25.277419,31.1,22.454839,37.5
92,2020,18.906667,09,27.1,20.956667,22.4,18.440000,32.7
93,2020,12.641935,10,61.4,15.280645,17.2,11.332258,73.1
94,2020,9.886667,11,39.7,12.043333,76.3,9.340000,18.1


In [242]:
# sort the order of columns

tree_regions_final = tree_regions_clean[['year', 'month', 'temp_rioja_mean', 'precip_rioja_sum', 'temp_catalunya_mean', 'precip_catalunya_sum', 'temp_duero_mean', 'precip_duero_sum']]
tree_regions_final

,year,month,temp_rioja_mean,precip_rioja_sum,temp_catalunya_mean,precip_catalunya_sum,temp_duero_mean,precip_duero_sum
0,2013,01,5.690323,126.9,7.509677,25.0,4.429032,74.7
1,2013,02,4.760714,89.2,7.310714,58.1,3.450000,60.2
2,2013,03,7.932258,132.2,10.283871,119.3,6.222581,138.8
3,2013,04,9.703333,57.3,12.816667,86.2,8.890000,60.0
4,2013,05,9.938710,86.0,14.525806,47.4,9.951613,59.1
...,...,...,...,...,...,...,...,...
91,2020,08,21.722581,20.8,25.277419,31.1,22.454839,37.5
92,2020,09,18.906667,27.1,20.956667,22.4,18.440000,32.7
93,2020,10,12.641935,61.4,15.280645,17.2,11.332258,73.1
94,2020,11,9.886667,39.7,12.043333,76.3,9.340000,18.1


In [243]:
# save clean dataset to tree_regions_clean.csv

tree_regions_final.to_csv("../data/tree_regions_final.csv", index=False)

In [244]:
# test if the file was saved correctly, by loading it

pd.read_csv("../data/tree_regions_final.csv")

,year,month,temp_rioja_mean,precip_rioja_sum,temp_catalunya_mean,precip_catalunya_sum,temp_duero_mean,precip_duero_sum
0,2013,1,5.690323,126.9,7.509677,25.0,4.429032,74.7
1,2013,2,4.760714,89.2,7.310714,58.1,3.450000,60.2
2,2013,3,7.932258,132.2,10.283871,119.3,6.222581,138.8
3,2013,4,9.703333,57.3,12.816667,86.2,8.890000,60.0
4,2013,5,9.938710,86.0,14.525806,47.4,9.951613,59.1
...,...,...,...,...,...,...,...,...
91,2020,8,21.722581,20.8,25.277419,31.1,22.454839,37.5
92,2020,9,18.906667,27.1,20.956667,22.4,18.440000,32.7
93,2020,10,12.641935,61.4,15.280645,17.2,11.332258,73.1
94,2020,11,9.886667,39.7,12.043333,76.3,9.340000,18.1


In [265]:
# load dataframe to sql

def load_dataframe_to_mysql(df, database, table, username, password, host='localhost', port=3306):
    """
    Load a Pandas DataFrame into a MySQL database table.
    
    Parameters:
    df (pandas.DataFrame): tree_regions_final
    database (str): 'vivino'
    table (str): 'meteo'
    username (str): 'root'
    password (str): 'DBDB2021'
    host (str, optional): The hostname or IP address of the MySQL server (default is 'localhost').
    port (int, optional): The port number to use to connect to the MySQL server (default is 3306).
    """
    # create a MySQL engine using sqlalchemy
    engine = create_engine(f'mysql://{username}:{password}@{host}:{port}/{database}')

    # write the dataframe to a SQL table
    tree_regions_final.to_sql(name=table, con=engine, if_exists='replace', index=False)

    # close the database connection
    engine.dispose()

In [269]:
load_dataframe_to_mysql(tree_regions_final, 'vivino', 'meteo', 'root', 'DBDB2021')